In [8]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import snscrape.modules.twitter as sntwitter
import json

In [2]:
# Création du dictionnaire
tag_dict = {
    "lec": {
        "teams": {
            "astralis",
            "excel",
            "fnatic",
            "g2",
            "madlions",
            "misfits",
            "rogue",
            "skgaming",
            "bds",
            "vitality"
        },
        "players" : {
            "astralis" : ["whiteknight","zanzarah","dajor","kobbe","promisq"],
            "excel": ["finn", "markoon", "nukeduck", "patrik","advienne"],
            "fnatic":["wunder", "razork", "humanoid", "upset","hylissang"],
            "g2":["bb", "brokenblade", "jankos", "caps", "flakked", "targamas", "targou"],
            "madlions":["armut", "elyoya", "reeker", "unforgiven", "kaiser"],
            "misfits":["hirit", "shlatan", "vetheo", "neon", "mersa"],
            "rogue":["odoamne", "odo", "malrang", "larssen", "comp", "trymbi"],
            "skgaming":["jenax", "gilius", "sertuss", "jezu", "treatz"],
            "bds":["adam", "cinkrof", "nuclearint", "xmatty", "matty", "limit"],
            "vitality":["alphari", "selfmade", "perkz", "carzzy", "labrov"]
        }
    },
    'lfl' : {
        "teams": {
            'kcorp',
            'gamersorigin',
            'solary',
            'gameward',
            'ldlc',
            'vitality',
            'misfits',
            'bds',
            'elyandra',
            'oplon',
        },
        "players" : {
            "kcorp" : ["cabochard","113","saken","rekkles","hantera"],
            "gamersorigin": ["vizicsacsi", "karimkt", "ronaldo", "smiley","enjawve"],
            "solary":["kio", "djoko", "scarlet", "asza","steelback"],
            "gameward":["melonik", "akabane", "czekoload", "innaxe", "kamilius"],
            "ldlc":["ragner", "yike", "eika", "exakick", "doss"],
            "vitality":["szygenda", "skeanz", "diplex", "jeskla", "jactroll"],
            "misfits":["irrelevant", "tynx", "czajek", "woolite", "vander"],
            "bds":["agresivoo", "sheo", "xico", "crownshot", "erdote"],
            "elyandra":["badlulu", "memento", "rangjun", "codysun", "raxo"],
            "oplon":["darlik", "bruness", "peng", "tiger", "absolute"]
        }
    }
}






    


In [3]:
def generate_search_list(tag_dict):
    list_of_search = []
    ligues = tag_dict.keys()
    for ligue in ligues:
        ligue_data = tag_dict[ligue]
        teams = ligue_data["teams"]
        players = ligue_data["players"]
        for team in teams:
            player_of_team = players[team]
            for player in player_of_team:
                search = player + " " + team
                list_of_search.append(search)
            list_of_search.append(team)
        list_of_search.append(ligue)
    return(list_of_search)

In [4]:
def twitter_scrapping(): 
    tweets_dict = {}
    id_list = []

    # Using TwitterSearchScraper to scrape data and append tweets to list
    for tag in generate_search_list(tag_dict):

        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(tag+' since:2022-01-21 until:2022-01-23').get_items()):
            if i>1:
                break
            tag_list = tag.split(" ")
            if tweet.id not in id_list :
                id_list.append(tweet.id)
                tweets_dict[tweet.id] = {"date" : tweet.date, "content" :tweet.content, "like" : tweet.likeCount, "reply" : tweet.replyCount, "retweet" : (tweet.retweetCount + tweet.quoteCount),"username" : tweet.user.username, "tag" : tag_list}
            else :
                current_list = tweets_dict[tweet.id]["tag"]
                new_list = list(set(current_list) | set(tag_list))
                tweets_dict[tweet.id]["tag"] = new_list

    # Creating a dataframe from the tweets list above
    tweets_df = pd.DataFrame(tweets_dict)
    return tweets_df.T

In [5]:
df = twitter_scrapping()

In [16]:
df

,date,content,like,reply,retweet,username,tag
1485031839639162889,2022-01-22 23:29:15+00:00,@cheesykuidaore @Trippple_B @Ejneberg111 @wund...,1,1,0,just_gebby,"[bb, g2]"
1484998869226299393,2022-01-22 21:18:15+00:00,@nvad_k @bb__g2 يارب,1,0,0,aly_38,"[bb, g2]"
1485018160428376064,2022-01-22 22:34:54+00:00,@Trippple_B @Ejneberg111 @wunderlol Imagine ki...,0,1,0,cheesykuidaore,"[brokenblade, g2]"
1484984034073710593,2022-01-22 20:19:18+00:00,"Y NO SE MUERE BROKENBLADE, INVENTEN NIÑOS INVE...",0,0,0,WHOKILLEDDOKI,"[brokenblade, g2]"
1485018789574062083,2022-01-22 22:37:24+00:00,Giga chad Selfmade STEALS baron from G2 Jankos...,2,0,1,10_Martial,"[jankos, g2]"
...,...,...,...,...,...,...,...
1484982581716299778,2022-01-22 20:13:31+00:00,@BDS_GrabbZ Agresivoo and crownie next week,2,0,0,IgneusLoL,"[agresivoo, bds]"
1484976224170651648,2022-01-22 19:48:16+00:00,"Ja tylko przypomnę, że w akademii BDS jest Agr...",3,0,0,JerryRejchel,"[bds, erdote, crownshot]"
1484966269069807619,2022-01-22 19:08:42+00:00,@Twisted_Chips C’est clair que la botlane BDS ...,0,2,0,snowwons_,"[erdote, bds]"
1485038276176236548,2022-01-22 23:54:50+00:00,Je vous jure que Xmatty/Limit même en LFL ça n...,7,1,0,ajdounnn,[lfl]


In [23]:
df2 = df.drop(df.columns[0],axis=1)
df2

,content,like,reply,retweet,username,tag
1485031839639162889,@cheesykuidaore @Trippple_B @Ejneberg111 @wund...,1,1,0,just_gebby,"[bb, g2]"
1484998869226299393,@nvad_k @bb__g2 يارب,1,0,0,aly_38,"[bb, g2]"
1485018160428376064,@Trippple_B @Ejneberg111 @wunderlol Imagine ki...,0,1,0,cheesykuidaore,"[brokenblade, g2]"
1484984034073710593,"Y NO SE MUERE BROKENBLADE, INVENTEN NIÑOS INVE...",0,0,0,WHOKILLEDDOKI,"[brokenblade, g2]"
1485018789574062083,Giga chad Selfmade STEALS baron from G2 Jankos...,2,0,1,10_Martial,"[jankos, g2]"
...,...,...,...,...,...,...
1484982581716299778,@BDS_GrabbZ Agresivoo and crownie next week,2,0,0,IgneusLoL,"[agresivoo, bds]"
1484976224170651648,"Ja tylko przypomnę, że w akademii BDS jest Agr...",3,0,0,JerryRejchel,"[bds, erdote, crownshot]"
1484966269069807619,@Twisted_Chips C’est clair que la botlane BDS ...,0,2,0,snowwons_,"[erdote, bds]"
1485038276176236548,Je vous jure que Xmatty/Limit même en LFL ça n...,7,1,0,ajdounnn,[lfl]


In [41]:
js = df.to_json(orient = 'records')

In [42]:
js2 = json.loads(js)

In [43]:
js2

[{'date': 1642894155000,
  'content': '@cheesykuidaore @Trippple_B @Ejneberg111 @wunderlol ?bb was arguably the best player of g2 in first games and rn faced a guy that also gapped Wunder,did Wunder played against any good toplaner this split besides Alphari to make a real impression about his form?and its not like he was kicked,he was sold to a top team,thats esports',
  'like': 1,
  'reply': 1,
  'retweet': 0,
  'username': 'just_gebby',
  'tag': ['bb', 'g2']},
 {'date': 1642886295000,
  'content': '@nvad_k @bb__g2 يارب',
  'like': 1,
  'reply': 0,
  'retweet': 0,
  'username': 'aly_38',
  'tag': ['bb', 'g2']},
 {'date': 1642890894000,
  'content': '@Trippple_B @Ejneberg111 @wunderlol Imagine kicking a player after one bad year lol... Wunder has been consistently good throughout the years, last year was a bump in the road but g2 got rid of him and got BrokenBlade which is much worse..',
  'like': 0,
  'reply': 1,
  'retweet': 0,
  'username': 'cheesykuidaore',
  'tag': ['brokenblade'

In [24]:
type(js2)

dict

In [32]:
client = MongoClient('localhost',27017)

In [33]:
print(client.list_database_names())

['admin', 'config', 'local']


In [27]:
db = client.project
collection = db['LoL_community']

In [38]:
db.collection.insert_one(df.to_dict())

InvalidDocument: documents must have only string keys, key was 1485031839639162889

In [30]:
 print(client.list_database_names())

['admin', 'config', 'local', 'project']


In [31]:
collection.find_one()

{'_id': ObjectId('61f1198a6bbd711c1f438aca'),
 'columns': ['date', 'content', 'like', 'reply', 'retweet', 'username', 'tag'],
 'data': [[1642894155000,
   '@cheesykuidaore @Trippple_B @Ejneberg111 @wunderlol ?bb was arguably the best player of g2 in first games and rn faced a guy that also gapped Wunder,did Wunder played against any good toplaner this split besides Alphari to make a real impression about his form?and its not like he was kicked,he was sold to a top team,thats esports',
   1,
   1,
   0,
   'just_gebby',
   ['bb', 'g2']],
  [1642886295000, '@nvad_k @bb__g2 يارب', 1, 0, 0, 'aly_38', ['bb', 'g2']],
  [1642890894000,
   '@Trippple_B @Ejneberg111 @wunderlol Imagine kicking a player after one bad year lol... Wunder has been consistently good throughout the years, last year was a bump in the road but g2 got rid of him and got BrokenBlade which is much worse..',
   0,
   1,
   0,
   'cheesykuidaore',
   ['brokenblade', 'g2']],
  [1642882758000,
   'Y NO SE MUERE BROKENBLADE, IN